## 카드 승인 내역 엑셀 처리
1단계로 카드승인내역에서 불필요한 라인, 불필요한 컬럼(가뱅점번호, 단말기번호) 삭제하기

In [1]:

# KICC 신용거래내역 엑셀 파일 테이터 처리

import pandas as pnds
import pickle
import datetime
import font_red_on_excel as font_red

# 1. 엑셀 파일을 읽어서 DataFrame 생성
filename = './신용거래내역조회_0618.xlsx'                 # KICC 신용거래내역 엑셀파일
card_history_df = pnds.read_excel(
                                filename, header=0, thousands = ',',           # 첫번쨰 라인은 컬럼명(header), 금액 천단위 구분자 ',' 고려
                                dtype={'거래고유번호': str,                         # 너무 큰 숫자들이므로 문자열로 셋팅
                                        '금액': 'int64',
                                        '승인번호': str}                            # 영문이나 숫자 '0'도 표시되어야 함으로 문자열 타입
                                )
# 2. 데이터 전처리
# 2-1. '거래일시' 컬럼명 수정
card_history_df.rename(columns={'거래일시▼': 'date'}, inplace=True)     # '거래일시' 컬럼명 'date'로 수정 => 다른 dataframe과 통일

# # 데이터 타입 변경
# card_history_df['금액'] = card_history_df['금액'].astype('int64', errors='ignore')      # '금액' : int64

# '거래고유번호' 컬럼에 있는 내용중에 NaN인 행 제거
card_history_df.dropna(subset=['거래고유번호'], inplace=True)

# 2-2. '승인번호' 중복 라인제거를 위한 중복 정보 추가
# dup_series = card_history_df.duplicated(['승인번호'], keep=False)           # '승인번호' 중복 여부 Bool series 리턴, 중복되면 모두 True
# card_history_df = pnds.concat([card_history_df, dup_series], axis=1)        # 결과를 dataframe column 으로 추가
# card_history_df.rename(columns={0: 'dup'}, inplace=True)                    # 추가된 중복 컬럼이름을 'dup'로 변경

card_history_df.drop_duplicates(subset=['승인번호'], keep=False, inplace=True)

# # 2-3. 중복 라인들을 제외하면서, 동시에 필요 컬럼들만 추출
# card_history_df = card_history_df.loc[card_history_df['dup'] == False][['거래고유번호', '승인구분', 'date', '카드번호', '발급카드사', '매입카드사', '금액']]
card_history_df = card_history_df[['거래고유번호', '승인구분', 'date', '카드번호', '발급카드사', '매입카드사', '금액', '할부개월', '승인번호']]

# card_history_df

# 이전 거래내역에서 이미 등록된 거래 내역은 제외
card_history_df = card_history_df[card_history_df['거래고유번호'].isin(font_red.read_red_color()) == False]

# 2-4. date 기준 Sorting
card_history_df.sort_values(by=['date'], inplace=True)

card_history_df

# # 2-5. date 포맷 변경: '%Y-%m-%d'
# card_history_df['date'] = card_history_df['date'].map(lambda str_data: str_data.split()[0], na_action='ignore') # 날짜 포맷 : '%Y-%m-%d'
# # 다른 방식으로서, 데이터타입을 datetime으로하고 date포맷을 변경하는 방법
# # opera_df['TRX_DATE'] = opera_df['TRX_DATE'].astype('datetime64[ns]', errors='ignore')    # date 포맷 변경을 위해서 datetimee 형으로 변환
# # opera_df['TRX_DATE'] = opera_df['TRX_DATE'].dt.strftime('%Y-%m-%d')     # 연산을 통해 포맷 변경 => 반환 타입은 일반 객체로 변경됨

# # card_history_df

# # DataFrame 저장
# target_date = (datetime.datetime.now() - datetime.timedelta(1)).strftime('%Y%m%d')  # 어제날짜의 폴더 이름
# base_dir = 'C:\\work\\pycard\\dtdata\\' + target_date
# now_date = card_history_df.iloc[1]['date']      # 정렬된 상태에서의 첫번쨰 인덱스를 선택 (정렬 이전의 지료에서 1번 인덱스는 다음날의 자료일 가능성이 있음)
# file_name = '/dt_kicc_history_' + now_date
# data_dir = base_dir + file_name
# with open(data_dir, "wb") as file:
#     pickle.dump(card_history_df, file)

57
./신용거래내역조회_220617.xlsx
42


c:\ProgramData\Anaconda3\envs\main\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,거래고유번호,승인구분,date,승인번호,카드번호,발급카드사,매입카드사,금액,할부개월
56,180033311224,승인,2022-06-18 00:15:01,68234355,4140-0307-6417-4926,IBK비씨카드,비씨카드,222090,일시불
55,180021057808,승인,2022-06-18 00:27:07,31261659,4265-8692-1246-4648,KB 기업카드,국민카드,69000,일시불
54,180012059674,승인,2022-06-18 00:28:25,704000,5557-9774-6919-9785,해외마스타,하나구외환,64152,일시불
53,180011061640,승인,2022-06-18 00:29:47,064643,5567-1717-7477-8230,해외마스타,하나구외환,48807,일시불
52,180011063512,승인,2022-06-18 00:31:04,934000,5557-9777-5444-8848,해외마스타,하나구외환,83309,일시불
51,180033321033,승인,2022-06-18 00:32:34,547335,5137-9680-0045-3830,해외마스타,하나구외환,198000,일시불
50,180012067480,승인,2022-06-18 00:33:57,910200,5557-9777-5279-3690,해외마스타,하나구외환,48114,일시불
49,180033322338,승인,2022-06-18 00:35:10,31261839,4265-8692-1246-4648,KB 기업카드,국민카드,75000,일시불
48,180022079838,승인,2022-06-18 00:41:37,691700,5557-9775-9201-8571,해외마스타,하나구외환,48114,일시불
47,180012081329,승인,2022-06-18 00:42:26,68375058,9430-3214-1229-0968,부산비씨카드,비씨카드,250000,일시불
